In [2]:
%matplotlib inline
#ESC + Z para recuperar celdas borradas!!!!!!!!!!

In [ ]:
#import sqlite3

# Crear una conexión a la base de datos (si no existe, se crea una nueva)
#datos = sqlite3.connect('basededatosHUBU.db')

# Crear una tabla
#datos.execute('''CREATE TABLE usuarios
                #(id INTEGER PRIMARY KEY,
                #nombre TEXT NOT NULL,
                #email TEXT NOT NULL);''')
# Cerrar la conexión
#datos.close()

Como Python no es un sistema de gestión de bases de datos, es necesario el uso de diferentes sistemas de gestión de datos como PostgreSQL para la creación de bases de datos. 
Para la creación es necesario elegir un sistema de gestión de bases de datos y luego usar la biblioteca de Python específica para interactuar. 


# CÓDIGO DEFINITIVO PRUEBA PDF INVENTADOS

In [164]:
import imageio as iio
import fitz 
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tabulate import tabulate
import tabula

In [165]:
# Usamos la biblioteca os para acceder al sistema de archivos y verificar cada archivo de la ruta. 
# Se crea una lista vacía llamada ficheros para almacenar los resultados y se usa la os.listdir(ruta) para obtener la lista de los archivos que se encuentran en esa ruta. 
# Con for leemos cada uno de los archivos pdf y lo añadimos a ficheros. 
# Al finalizar, tenemos una lista con todos los archivos de esa ruta. 
def LeerFicherosPDF(ruta):
    ficheros = []
    contenido = os.listdir(ruta)
    for fichero in contenido:
        if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.pdf'):
            ficheros.append(fichero)
    return ficheros

In [166]:
def LeerDocumento(nombreFichero):
    with fitz.open(nombreFichero) as doc:
        text=""
        for page in doc:
            text = text + page.get_text()
    return text.split('\n')


In [167]:
def BuscarValor(textoBuscar, lines):
    Encontrados = []

    valores = [1 if re.search(textoBuscar, line) else 0 for line in lines]
    valores = [i for i, s in enumerate(valores) if s==1 in valores]

    posiciones = len(textoBuscar)
    
    for i in valores:
        Encontrados.append(lines[i][lines[i].rfind(textoBuscar)+posiciones:].strip())

    return Encontrados

In [168]:
def GenerarImagen(ruta, fichero):
    i = 0   
    zoom = 4
    mat = fitz.Matrix(zoom, zoom)
    with fitz.open(os.path.join(ruta,fichero)) as doc:
        for page in doc:
            i+=1
            val = f"image_{i+1}.png"
            pix = page.get_pixmap(matrix=mat)
            pix.save(os.path.join(ruta,fichero + "_" + val))
            #plt.figure(figsize=(8,15))
            #img = iio.imread(val)
            #plt.imshow(img, cmap='gray')
            #plt.show()

In [169]:
diagnostico = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/Diagnostico.xlsx")
print(diagnostico)

#Creo un diccionario con los valores tanto del diagnóstico como del número correspondiente. 
diagnosticos_dic = dict(zip(diagnostico["DIAGNÓSTICO"], diagnostico["NÚMERO DIAGNÓSTICO"]))

for diagnostico in diagnosticos_dic:
    valor = diagnosticos_dic.get(diagnostico)
    print(diagnostico, valor)


                                          DIAGNÓSTICO  NÚMERO DIAGNÓSTICO
0                         Carcinoma del tracto biliar                 1.0
1                                   Colangiocarcinoma                 1.1
2                     Carcinoma de la vesícula biliar                 1.2
3                                    Cáncer de vejiga                 2.0
4                       Carcinoma vesicular uroterial                 2.1
5                                      Cáncer de mama                 3.0
6                             Triple negativo de mama                 3.1
7              Neoplasia del sistema nervioso central                 4.0
8                                              Glioma                 4.1
9                                     Cáncer cervical                 5.0
10                                      Condrosarcoma                 6.0
11                                 Cáncer colorrectal                 7.0
12            Carcinoma de células esc

In [170]:
genes = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/Genes.xlsx")
print(genes)

mutaciones = genes["GEN"].unique()
#print(mutaciones)
#type(mutacion)

       GEN  Número gen
0     ABL1           1
1     AKT1           2
2     AKT3           3
3      ALK           4
4       AR           5
5      AXL           6
6     BRAF           7
7    CCND1           8
8     CDK4           9
9     CDK6          10
10  CTNNB1          11
11    DDR2          12
12    EGFR          13
13   ERBB2          14
14   ERBB3          15
15   ERBB4          16
16     ERG          17
17    ESR1          18
18    ETV1          19
19    ETV4          20
20    ETV5          21
21   FGFR1          22
22   FGFR2          23
23   FGFR3          24
24   FGFR4          25
25   GNA11          26
26    GNAQ          27
27    HRAS          28
28    IDH1          29
29    IDH2          30
30    JAK1          31
31    JAK2          32
32    JAK3          33
33     KIT          34
34    KRAS          35
35  MAP2K1          36
36  MAP2K2          37
37     MET          38
38    MTOR          39
39     MYC          40
40    MYCN          41
41    NRAS          42
42   NTRK1 

In [171]:
#Creo un diccionario con los valores del 
mutaciones_dic = dict(zip(genes["GEN"], genes["Número gen"]))
mutaciones_dic

for gen, valor in mutaciones_dic.items():
    print(gen, valor)

ABL1 1
AKT1 2
AKT3 3
ALK 4
AR 5
AXL 6
BRAF 7
CCND1 8
CDK4 9
CDK6 10
CTNNB1 11
DDR2 12
EGFR 13
ERBB2 14
ERBB3 15
ERBB4 16
ERG 17
ESR1 18
ETV1 19
ETV4 20
ETV5 21
FGFR1 22
FGFR2 23
FGFR3 24
FGFR4 25
GNA11 26
GNAQ 27
HRAS 28
IDH1 29
IDH2 30
JAK1 31
JAK2 32
JAK3 33
KIT 34
KRAS 35
MAP2K1 36
MAP2K2 37
MET 38
MTOR 39
MYC 40
MYCN 41
NRAS 42
NTRK1 43
NTRK2 44
NTRK3 45
PDGFRA 46
PIK3CA 47
PPARG 48
RAF1 49
RET 50
ROS1 51
SMO 52


In [172]:
genes = pd.read_excel("C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/Genes.xlsx")
#print(genes)

mutacion = genes["GEN"].unique()
a = list(mutacion)
print(mutacion)
type(mutacion)
print(a)
type(a)


['ABL1' 'AKT1' 'AKT3' 'ALK' 'AR' 'AXL' 'BRAF' 'CCND1' 'CDK4' 'CDK6'
 'CTNNB1' 'DDR2' 'EGFR' 'ERBB2' 'ERBB3' 'ERBB4' 'ERG' 'ESR1' 'ETV1' 'ETV4'
 'ETV5' 'FGFR1' 'FGFR2' 'FGFR3' 'FGFR4' 'GNA11' 'GNAQ' 'HRAS' 'IDH1'
 'IDH2' 'JAK1' 'JAK2' 'JAK3' 'KIT' 'KRAS' 'MAP2K1' 'MAP2K2' 'MET' 'MTOR'
 'MYC' 'MYCN' 'NRAS' 'NTRK1' 'NTRK2' 'NTRK3' 'PDGFRA' 'PIK3CA' 'PPARG'
 'RAF1' 'RET' 'ROS1' 'SMO']
['ABL1', 'AKT1', 'AKT3', 'ALK', 'AR', 'AXL', 'BRAF', 'CCND1', 'CDK4', 'CDK6', 'CTNNB1', 'DDR2', 'EGFR', 'ERBB2', 'ERBB3', 'ERBB4', 'ERG', 'ESR1', 'ETV1', 'ETV4', 'ETV5', 'FGFR1', 'FGFR2', 'FGFR3', 'FGFR4', 'GNA11', 'GNAQ', 'HRAS', 'IDH1', 'IDH2', 'JAK1', 'JAK2', 'JAK3', 'KIT', 'KRAS', 'MAP2K1', 'MAP2K2', 'MET', 'MTOR', 'MYC', 'MYCN', 'NRAS', 'NTRK1', 'NTRK2', 'NTRK3', 'PDGFRA', 'PIK3CA', 'PPARG', 'RAF1', 'RET', 'ROS1', 'SMO']


list

## ENSAYOS CLÍNICOS Y TRATAMIENTOS DISPONIBLES. 

In [272]:
patron = r"(\d+)\s* Ensayos clínicos"
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
ficheros = LeerFicherosPDF(Ruta)

lista_ensayos = []
ensayos_finales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    
    ensayos = 0
    for line in lines:
        resultado = re.search(patron, line)
        if resultado:
            ensayos = int(resultado.group(1))
            
    lista_ensayos.append(ensayos)   
    
    print(ficheroPDF + " -> Ensayos: " + str(ensayos))
print(lista_ensayos)


for i in lista_ensayos:
    if i == 0:
        ensayos_finales.append(0)
    else:
        ensayos_finales.append(1)
        
print(ensayos_finales)


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf -> Ensayos: 6
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf -> Ensayos: 5
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf -> Ensayos: 0
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf -> Ensayos: 0
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf -> Ensayos: 3
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf -> Ensayos: 5
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf -> Ensayos: 1
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf -> Ensayos: 0
[6, 5, 0, 0, 3, 5, 1, 0]
[1, 1, 0, 0, 1, 1, 1, 0]


In [174]:
patron2 = r"(\d+)\s* Tratamientos disponibles"
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
ficheros = LeerFicherosPDF(Ruta)

lista_tratamientos = []
tratamientos_finales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    tratamientos = 0
    
    for line in lines:
        resultado = re.search(patron2, line)
        if resultado:
            tratamientos = int(resultado.group(1))
    
    lista_tratamientos.append(tratamientos)
    print(ficheroPDF + " -> Tratamientos: " + str(tratamientos))
    
print(lista_tratamientos)


for i in lista_tratamientos:
    if i == 0:
        tratamientos_finales.append(0)
    else:
        tratamientos_finales.append(1)
print(tratamientos_finales)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf -> Tratamientos: 6
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf -> Tratamientos: 1
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf -> Tratamientos: 0
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf -> Tratamientos: 0
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf -> Tratamientos: 3
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf -> Tratamientos: 0
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf -> Tratamientos: 0
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf -> Tratamientos: 1
[6, 1, 0, 0, 3, 0, 0, 1]
[1, 1, 0, 0, 1, 0, 0, 1]


## MUTACIONES Y DERIVADOS

In [175]:
#Todos los genes mutados que hay en cada ficherO (ESTO NO ME INTERSA!)
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'

ficheros = LeerFicherosPDF(Ruta)

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    
    for mutacion in mutaciones:
        if mutacion in lines:
            print(ficheroPDF + " - Existe: " + mutacion)
            

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: ALK
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: FGFR4
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: ALK
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: FGFR4
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: MET
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Existe: ALK
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Existe: FGFR4
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: ALK
Sample_4_v100_03eb528b-

In [177]:
#Genes mutados de interés (PATOGENICOS / VACIOS ) BIEN!!!
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'

ficheros = LeerFicherosPDF(Ruta)

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
                
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            #print(ficheroPDF + " - Existe: " + mutacion)
            #print(posiciones)
            if mutacion == "FGFR4": #solo el tipo p.(P136L)
                #posicion = lines.index("FGFR4")
                #print(posicion)
                #print(posicion + 1)
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    print(ficheroPDF + " - Existe: " + mutacion)
            else:
                benigno = False
                #print(mutacion)
                for a in range(posicion+1, posicion+10):
                    #print(str(a) + " > " + lines[a])
                    if "Benign" in lines[a]:
                        #print(lines[a])
                        benigno = True
                if benigno == False:
                    print(ficheroPDF + " - Existe: " + mutacion)

                

#            elif (posiciones<len(lines) - 9 and lines[posiciones + 9] == "Benign"):
 #               pass
  #          elif (posiciones<len(lines) - 9 and lines[posiciones + 9] == "Benign/Likelybenign"):
   #             pass
    #        else: 
     #           print(ficheroPDF + " - Existe: " + mutacion)
      #          #print(posiciones)
                
#FALLOS.
# NO APARECEN LOS REPETIDOS DOS VECES, SOLO APARECEN UNA solucion: ID


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: PIK3CA
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: CDK6
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: MET
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: EGFR
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf - Existe: HRAS
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf - Existe: KIT
Sample_8_v100_1445

In [178]:
#Solo genes patogénicos BIEN!!!
for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)

            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
                

Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1 - Pathogenic


In [179]:
patron_frecuencia = re.compile(r"\d{2}\.\d{2}%")
frecuenciasPato = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    print(ficheroPDF + "- Existe: " + mutacion + "- Pathogenic")
            
                    for i in lines[posicion:posicion+10]:
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            #frecuencias.append(frec)
    frecuenciasPato.extend(lista_frec)
                            #print(f"   Frecuencia: {frec}")
        
    print(f"{ficheroPDF} - Frecuencias: {lista_frec}")
    
frecuenciasPato


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Frecuencias: []
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf- Existe: KRAS- Pathogenic
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Frecuencias: ['66.50%']
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Frecuencias: []
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Frecuencias: []
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Frecuencias: []
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf- Existe: CTNNB1- Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Frecuencias: ['27.51%']
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf - Frecuencias: []
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-0

['66.50%', '27.51%']

In [180]:
patogen = {}

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    genpato2 = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    genpato2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
            
                    
    
    patogen[ficheroPDF] = genpato2
    
print(patogen)

Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1 - Pathogenic
{'Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf': [], 'Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf': ['KRAS'], 'Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf': [], 'Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf': [], 'Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf': [], 'Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf': ['CTNNB1'], 'Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf': [], 'Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf': []}


In [181]:
patologicos = list(patogen.values())
patologicos

[[], ['KRAS'], [], [], [], ['CTNNB1'], [], []]

In [182]:
numero_iden_pato = []
for i in patologicos:
    valores = [mutaciones_dic.get(gen, 0) for gen in i]
    numero_iden_pato.append(valores)

print(numero_iden_pato)

[[], [35], [], [], [], [11], [], []]


In [183]:
#MEZCLA DE AMBOS
for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
                
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            #print(ficheroPDF + " - Existe: " + mutacion)
            #print(posiciones)
            if mutacion == "FGFR4": #solo el tipo p.(P136L)
                #posicion = lines.index("FGFR4")
                #print(posicion)
                #print(posicion + 1)
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    print(ficheroPDF + " - Existe: " + mutacion)
            else:
                benigno = False
                #print(mutacion)
                for a in range(posicion+1, posicion+10):
                    #print(str(a) + " > " + lines[a])
                    if "Benign" in lines[a]:
                        #print(lines[a])
                        benigno = True
                if benigno == False:
                    print(ficheroPDF + " - Existe: " + mutacion)
                    
            for a in range(posicion+1, posicion+10):
                if "Pathogeni" in lines[a]:
                    print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
                    
patogen = {}

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    genpato2 = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    genpato2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion + " - Pathogenic")
            
                    
    
    patogen[ficheroPDF] = genpato2
    
print(patogen)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS - Pathogenic
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: PIK3CA
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: CDK6
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: MET
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Existe: CTNNB1 - Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Ex

In [184]:
#TOTAL DEL NÚMERO DE MUTACIONES BIEN!!
ficheros = LeerFicherosPDF(Ruta)
max_mut = 0

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    total_mut = 0
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            #print(ficheroPDF + " - Existe: " + mutacion)
            #print(posiciones)
            if mutacion == "FGFR4": #solo el tipo p.(P136L)
                #posicion = lines.index("FGFR4")
                #print(posicion)
                #print(posicion + 1)
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    total_mut += 1
                    print(ficheroPDF + " - Existe: " + mutacion)
            else:
                benigno = False
                #print(mutacion)
                for a in range(posicion+1, posicion+10):
                    #print(str(a) + " > " + lines[a])
                    if "Benign" in lines[a]:
                        #print(lines[a])
                        benigno = True
                if benigno == False:
                    total_mut += 1
                    print(ficheroPDF + " - Existe: " + mutacion)
                    
                    
    if total_mut > max_mut:
        max_mut = total_mut
    print(f"{ficheroPDF} - Número total de mutaciones : {total_mut}")


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Número total de mutaciones : 1
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Número total de mutaciones : 1
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Número total de mutaciones : 0
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: PIK3CA
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Número total de mutaciones : 2
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: CDK6
Sample_5_v100_da86c58e-d529-400b-8812-506948

## MUTACIONES 

In [186]:
#OBTENCIÓN GENES, NÚMERO Y VALORES BIEN!!
ficheros = LeerFicherosPDF(Ruta)
max_mut = 0
genes_mut2 = {}

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))
    total_mut = 0
    encontrados2 = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            #print(ficheroPDF + " - Existe: " + mutacion)
            #print(posiciones)
            if mutacion == "FGFR4": #solo el tipo p.(P136L)
                #posicion = lines.index("FGFR4")
                #print(posicion)
                #print(posicion + 1)
                if (posicion < len(lines)-1 and lines[posicion + 1] == "p.(P136L)"):
                    pass
                else:
                    total_mut += 1
                    encontrados2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion)
            else:
                benigno = False
                #print(mutacion)
                for a in range(posicion+1, posicion+10):
                    #print(str(a) + " > " + lines[a])
                    if "Benign" in lines[a]:
                        #print(lines[a])
                        benigno = True
                if benigno == False:
                    total_mut += 1
                    encontrados2.append(mutacion)
                    print(ficheroPDF + " - Existe: " + mutacion)
                    
    
    genes_mut2[ficheroPDF] = encontrados2
    if total_mut > max_mut:
        max_mut = total_mut
    print(f"{ficheroPDF} - Número total de mutaciones : {total_mut}")
print(genes_mut2)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Número total de mutaciones : 1
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Número total de mutaciones : 1
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Número total de mutaciones : 0
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: PIK3CA
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Número total de mutaciones : 2
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: CDK6
Sample_5_v100_da86c58e-d529-400b-8812-506948

In [187]:
mut = list(genes_mut2.values())
mut

[['MYC'],
 ['KRAS'],
 [],
 ['FGFR1', 'PIK3CA'],
 ['CDK6', 'MET'],
 ['CTNNB1', 'EGFR'],
 ['HRAS'],
 ['KIT', 'MYC']]

In [188]:
num_mutaciones = [len(i) for i in mut]
print(num_mutaciones)

[1, 1, 0, 2, 2, 2, 1, 2]


In [189]:
numero_iden = []
for i in mut:
    valores = [mutaciones_dic.get(gen, 0) for gen in i]
    numero_iden.append(valores)

print(numero_iden)

[[40], [35], [], [22, 47], [10, 38], [11, 13], [28], [34, 40]]


## FRECUENCIA ALÉLICA

In [270]:
patron_frecuencia = re.compile(r"\d{2}\.\d{2}")
frecuencias_totales = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            if mutacion == "FGFR4":
                if posicion < len(lines) - 1 and lines[posicion + 1] == "p.(P136L)":
                    pass
                else:
                    print(f"{ficheroPDF} - Existe: {mutacion}")
            else:
                benigno = False
                for a in range(posicion + 1, posicion + 10):
                    if "Benign" in lines[a]:
                        benigno = True
                if not benigno:
                    print(f"{ficheroPDF} - Existe: {mutacion}")
                    for i in lines[posicion:posicion+10]:
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            frecuencias.append(frec)
        
    print(f"{ficheroPDF} - Frecuencias: {lista_frec}")
    frecuencias_totales.append(lista_frec)
    
print(frecuencias_totales)


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Existe: MYC
Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Frecuencias: ['49.56']
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Existe: KRAS
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Frecuencias: ['66.50']
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Frecuencias: []
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: FGFR1
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Existe: PIK3CA
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Frecuencias: []
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Existe: CDK6
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Exist

In [253]:
frecuencias_totales

[['49.56'],
 ['66.50'],
 [],
 [],
 ['21.75', '21.75'],
 ['27.51', '35.40'],
 ['46.17'],
 ['53.12', '60.64']]

In [254]:
patron_frecuencia = re.compile(r"\d{2}\.\d{2}")
frecuenciasPato = []

for ficheroPDF in ficheros:
    lines = LeerDocumento(os.path.join(Ruta, ficheroPDF))
    lista_frec = []
    
    for mutacion in mutaciones:
        if mutacion in lines:
            posicion = lines.index(mutacion)
            
            for a in range(posicion+1, min(posicion+10, len(lines))):
                if "Pathogeni" in lines[a]:
                    print(ficheroPDF + "- Existe: " + mutacion + "- Pathogenic")
            
                    for i in lines[posicion:posicion+10]:
                        resultado = re.search(patron_frecuencia, i)
                        if resultado:
                            frec = resultado.group()
                            lista_frec.append(frec)
                            #frecuencias.append(frec)
    frecuenciasPato.append(lista_frec)
                            #print(f"   Frecuencia: {frec}")
        
    print(f"{ficheroPDF} - Frecuencias: {lista_frec}")
    
print(frecuenciasPato)


Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf - Frecuencias: []
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf- Existe: KRAS- Pathogenic
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf - Frecuencias: ['66.50']
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf - Frecuencias: []
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf - Frecuencias: []
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf - Frecuencias: []
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf- Existe: CTNNB1- Pathogenic
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf - Frecuencias: ['27.51']
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf - Frecuencias: []
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-

In [255]:
frecuenciasPato

[[], ['66.50'], [], [], [], ['27.51'], [], []]

## ID VARIANTE

## NÚMERO DE CHIP Y PACIENTE

In [193]:
#Leemos todos los nombres de los archivos PDF
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
archivos = os.listdir(Ruta)

for archivo in archivos:
    if os.path.isfile(os.path.join(Ruta, archivo)) and archivo.endswith(".pdf"):
        print(archivo)

Sample_1_v100_ec96983e-d816-4baf-8544-ca0572acc9fa_2023-03-02_11-57-06-352_All.pdf
Sample_2_v100_dc008f2f-9bc7-4ac2-89d2-7f3ba6fdb04d_2023-03-02_11-56-10-540_All.pdf
Sample_3_v100_50c27cd1-3ee7-490c-8ec6-30091039bb9c_2023-03-02_11-55-40-900_All.pdf
Sample_4_v100_03eb528b-ded2-4db9-81fe-e1553385db88_2023-03-02_11-54-08-761_All.pdf
Sample_5_v100_da86c58e-d529-400b-8812-506948d0c3ed_2023-03-02_11-53-07-777_All.pdf
Sample_6_v100_52c1af10-c6e1-4f44-85d8-bfa7260fb9ff_2023-03-02_11-52-19-927_All.pdf
Sample_7_v100_869cc724-f5f4-4d61-a708-c818a6c062ef_2023-03-02_11-50-41-677_All.pdf
Sample_8_v100_14451fbc-44ce-480e-9107-c6ab537a1fda_2023-03-02_11-49-41-641_All.pdf


In [195]:
#Sacamos el tercer valor de cada nombre, que es el que indica el número del paciente
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
archivos = os.listdir(Ruta)
numero_paciente = []

for archivo in archivos:
    if os.path.isfile(os.path.join(Ruta, archivo)) and archivo.endswith(".pdf"):
        paciente = os.path.splitext(archivo)[0]
        pacientes = paciente[7]
        numero_paciente.append(pacientes)
        print(numero_paciente)

['1']
['1', '2']
['1', '2', '3']
['1', '2', '3', '4']
['1', '2', '3', '4', '5']
['1', '2', '3', '4', '5', '6']
['1', '2', '3', '4', '5', '6', '7']
['1', '2', '3', '4', '5', '6', '7', '8']


In [196]:
numero_paciente

['1', '2', '3', '4', '5', '6', '7', '8']

In [197]:
#Otra forma usando expresiones regulares para sacar el número de chip
Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
archivos = os.listdir(Ruta)
chip2 = []

for archivo in archivos:
    if os.path.isfile(os.path.join(Ruta, archivo)) and archivo.endswith(".pdf"):
        patron = r"v(\d+)_"
        resultado = re.search(patron, archivo)
        if resultado:
            numero_chip = resultado.group(1)
            chip2.append(numero_chip)
print(chip2)

['100', '100', '100', '100', '100', '100', '100', '100']


In [198]:
chip2

['100', '100', '100', '100', '100', '100', '100', '100']

## OBTENCIÓN DE ALGUNOS RESULTADOS.

In [199]:
fecha_Data = []
NHC_Data = [] 
Nbiopsia_Data = [] 
texto_Data = []

Ruta = 'C:/Users/luciv/OneDrive/Documentos/TFG_Lucia/RE_PDF_inventados'
#RutaPatricia = ... carpeta que nos hemos inventado para trabajar cuando se lo pase

ficheros = LeerFicherosPDF(Ruta)

for ficheroPDF in ficheros:
    #print(os.path.join(Ruta,ficheroPDF))
    lines = LeerDocumento(os.path.join(Ruta,ficheroPDF))

    #print(lines)

    NHC_Data.append(BuscarValor("NHC:", lines))
    Nbiopsia_Data.append(BuscarValor("biopsia:", lines))
    fecha_Data.append(BuscarValor("Fecha:", lines))
    texto_Data.append(BuscarValor("de la muestra:", lines))
   

    GenerarImagen(Ruta,ficheroPDF)

print(NHC_Data)
print(Nbiopsia_Data)
print(fecha_Data)
print(texto_Data)


[['00021', '00021', '00021'], ['00022', '00022', '00022'], ['00023', '00023'], ['00024', '00024'], ['00025', '00025', '00025'], ['00026', '00026', '00026'], ['00027', '00027'], ['00028', '00028']]
[['23B00021-A1', '23B00021-A1', '23B00021-A1'], ['22B00022-A2', '22B00022-A2', '22B00022-A2'], ['21B00023-A1', '21B00023-A1'], ['23B00024-A1', '23B00024-A1'], ['23C00025-A1', '23C00025-A1', '23C00025-A1'], ['23P00026', '23P00026', '23P00026'], ['22B00027-C4', '22B00027-C4'], ['23B00028-A2', '23B00028-A2']]
[['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023']]
[['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar 

## DIAGNÓSTICO.

In [200]:
textoDiag = []
numeroDiag = []

for i in texto_Data:
    sinduplicados = list(set(i))
    textoDiag.append([x for x in i if x in sinduplicados][0])
print(textoDiag)
    
for diagnostico in textoDiag:
    valor = diagnosticos_dic.get(diagnostico)
    numeroDiag.append(valor)
    print(numeroDiag)
    

['Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Cáncer tiroideo', 'Cáncer gástrico']
[15.1]
[15.1, 15.1]
[15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0]
[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0]


In [201]:
numeroDiag

[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0]

## NHC

In [202]:
NHC_Data_val = list(set([valor for sublist in NHC_Data for valor in sublist]))
print(NHC_Data_val)
NHC_final = sorted(NHC_Data_val)
print(NHC_final)

['00026', '00025', '00021', '00024', '00022', '00023', '00027', '00028']
['00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028']


## BIOPSIA

In [203]:
Nbiopsia_Data

[['23B00021-A1', '23B00021-A1', '23B00021-A1'],
 ['22B00022-A2', '22B00022-A2', '22B00022-A2'],
 ['21B00023-A1', '21B00023-A1'],
 ['23B00024-A1', '23B00024-A1'],
 ['23C00025-A1', '23C00025-A1', '23C00025-A1'],
 ['23P00026', '23P00026', '23P00026'],
 ['22B00027-C4', '22B00027-C4'],
 ['23B00028-A2', '23B00028-A2']]

In [204]:
lista_resultante =  []
elementos_vistos = set()

for sublist in Nbiopsia_Data:
    sublist_sin_duplicados = []
    for elemento in sublist:
        if elemento not in elementos_vistos:
            sublist_sin_duplicados.append(elemento)
            elementos_vistos.add(elemento)
    lista_resultante.append(sublist_sin_duplicados)

print(lista_resultante)

NB_values = [elemento for sublist in lista_resultante for elemento in sublist]
print(NB_values)
 

[['23B00021-A1'], ['22B00022-A2'], ['21B00023-A1'], ['23B00024-A1'], ['23C00025-A1'], ['23P00026'], ['22B00027-C4'], ['23B00028-A2']]
['23B00021-A1', '22B00022-A2', '21B00023-A1', '23B00024-A1', '23C00025-A1', '23P00026', '22B00027-C4', '23B00028-A2']


In [205]:
biopsia = [x[2] for x in NB_values]
biopsia

['B', 'B', 'B', 'B', 'C', 'P', 'B', 'B']

In [206]:
#B de biopsia = 1
#C de citología = 3
#P de punción = 2
B = "1"
C = "3"
P = "2"
Biopsia_solida = []

for i in biopsia:
    if i == "B":
        Biopsia_solida.append(B)
        print("1")
    elif i == "P":
        Biopsia_solida.append(P)
        print("2")
    else:
        Biopsia_solida.append(C)
        print("3")
        
print(Biopsia_solida)

1
1
1
1
3
2
1
1
['1', '1', '1', '1', '3', '2', '1', '1']


## FECHA

In [207]:
fechas = []
for i in fecha_Data:
    sinduplicados = list(set(i))
    fechas.append([x for x in i if x in sinduplicados][0])

print(fechas)


['24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023']


## VARIABLES DE INTERÉS QUE ALMACENO EN LOS DATAFRAMES

In [208]:
# Número de chip 
chip2

['100', '100', '100', '100', '100', '100', '100', '100']

In [209]:
# Número de paciente 
numero_paciente

['1', '2', '3', '4', '5', '6', '7', '8']

In [210]:
# NHC
NHC_final

['00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028']

In [211]:
# Número de biopsia 
NB_values

['23B00021-A1',
 '22B00022-A2',
 '21B00023-A1',
 '23B00024-A1',
 '23C00025-A1',
 '23P00026',
 '22B00027-C4',
 '23B00028-A2']

In [212]:
# Biopsia sólida 
Biopsia_solida

['1', '1', '1', '1', '3', '2', '1', '1']

In [213]:
# Fecha de informe
print(fecha_Data)
print("---------------------------------------------------------------------------------------------------------------------")
print(fechas)

[['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023'], ['24-mar-2023', '24-mar-2023']]
---------------------------------------------------------------------------------------------------------------------
['24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023', '24-mar-2023']


In [214]:
# Diagnóstico 
print(texto_Data)
print("------------------------------------------------------------------------------------------------------------------")
print(textoDiag)

[['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Carcinoma pulmonar no microcítico'], ['Cáncer tiroideo'], ['Cáncer gástrico']]
------------------------------------------------------------------------------------------------------------------
['Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Carcinoma pulmonar no microcítico', 'Cáncer tiroideo', 'Cáncer gástrico']


In [215]:
# Número de diagnóstico 
numeroDiag

[15.1, 15.1, 15.1, 15.1, 15.1, 15.1, 30.0, 11.0]

In [216]:
# Total del número de mutaciones
num_mutaciones

[1, 1, 0, 2, 2, 2, 1, 2]

In [217]:
# Mutaciones detectadas totales 
mut

[['MYC'],
 ['KRAS'],
 [],
 ['FGFR1', 'PIK3CA'],
 ['CDK6', 'MET'],
 ['CTNNB1', 'EGFR'],
 ['HRAS'],
 ['KIT', 'MYC']]

In [256]:
# % de frecuencia alélica
frecuencias_totales

[['49.56'],
 ['66.50'],
 [],
 [],
 ['21.75', '21.75'],
 ['27.51', '35.40'],
 ['46.17'],
 ['53.12', '60.64']]

In [257]:
# % frecuencias alélicas patológicas 
frecuenciasPato

[[], ['66.50'], [], [], [], ['27.51'], [], []]

In [219]:
# Número de lecturas (% de frecuencia alélica en ARN) 


In [ ]:
# Número de copias (% de frecuencia alélica en copias)


In [220]:
# Nº de la mutación específica detectada 
numero_iden

[[40], [35], [], [22, 47], [10, 38], [11, 13], [28], [34, 40]]

In [221]:
# Ensayos clínicos 
print(lista_ensayos)
print("-----------------------------------------------------------------------------------------------------------------")
print(ensayos_finales)

[6, 5, 0, 0, 3, 5, 1, 0]
-----------------------------------------------------------------------------------------------------------------
[1, 1, 0, 0, 1, 1, 1, 0]


In [222]:
# Fármaco aprobado 
print(lista_tratamientos)
print("-----------------------------------------------------------------------------------------------------------------")
print(tratamientos_finales)

[6, 1, 0, 0, 3, 0, 0, 1]
-----------------------------------------------------------------------------------------------------------------
[1, 1, 0, 0, 1, 0, 0, 1]


## CREACIÓN DE LOS DATAFRAMES. 

In [223]:
Tabla1Pacientes = pd.DataFrame({'Número de chip': chip2, 'Número de paciente': numero_paciente, 'NHC': NHC_final, 'Número de biopsia': NB_values, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fechas, 'Diagnóstico': textoDiag, 'Mutaciones detectadas':mut})

In [224]:
Tabla1Pacientes

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Mutaciones detectadas
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,[MYC]
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,[KRAS]
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,[]
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"[FGFR1, PIK3CA]"
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,"[CDK6, MET]"
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,"[CTNNB1, EGFR]"
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,[HRAS]
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,"[KIT, MYC]"


In [225]:
Tabla2Diagnostico = pd.DataFrame({'Número de chip': chip2, 'Número de paciente': numero_paciente, 'Diagnóstico': textoDiag, 'Número del diagnóstico': numeroDiag})

In [226]:
Tabla2Diagnostico

,Número de chip,Número de paciente,Diagnóstico,Número del diagnóstico
0,100,1,Carcinoma pulmonar no microcítico,15.1
1,100,2,Carcinoma pulmonar no microcítico,15.1
2,100,3,Carcinoma pulmonar no microcítico,15.1
3,100,4,Carcinoma pulmonar no microcítico,15.1
4,100,5,Carcinoma pulmonar no microcítico,15.1
5,100,6,Carcinoma pulmonar no microcítico,15.1
6,100,7,Cáncer tiroideo,30.0
7,100,8,Cáncer gástrico,11.0


In [229]:
Tabla4MutacionesEspecificas = pd.DataFrame({'Mutaciones detectadas':mut, 'Número de la mutación específica':numero_iden, 'Total del número de mutaciones': num_mutaciones, })

In [230]:
Tabla4MutacionesEspecificas

,Mutaciones detectadas,Número de la mutación específica,Total del número de mutaciones
0,[MYC],[40],1
1,[KRAS],[35],1
2,[],[],0
3,"[FGFR1, PIK3CA]","[22, 47]",2
4,"[CDK6, MET]","[10, 38]",2
5,"[CTNNB1, EGFR]","[11, 13]",2
6,[HRAS],[28],1
7,"[KIT, MYC]","[34, 40]",2


In [258]:
Tabla5Patologicas = pd.DataFrame({'Genes patológicos': patologicos, 'Número de la mutación específica':numero_iden_pato, '% frecuencia alélica':frecuenciasPato})

In [259]:
Tabla5Patologicas

,Genes patológicos,Número de la mutación específica,% frecuencia alélica
0,[],[],[]
1,[KRAS],[35],[66.50]
2,[],[],[]
3,[],[],[]
4,[],[],[]
5,[CTNNB1],[11],[27.51]
6,[],[],[]
7,[],[],[]


In [227]:
Tabla6Informacion = pd.DataFrame({'Diagnóstico': textoDiag, 'Ensayos clínicos': lista_ensayos,'SI/NO ensayo':ensayos_finales, 'Fármaco aprobado': lista_tratamientos, 'SI/NO fármacos': tratamientos_finales})

In [228]:
Tabla6Informacion

,Diagnóstico,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos
0,Carcinoma pulmonar no microcítico,6,1,6,1
1,Carcinoma pulmonar no microcítico,5,1,1,1
2,Carcinoma pulmonar no microcítico,0,0,0,0
3,Carcinoma pulmonar no microcítico,0,0,0,0
4,Carcinoma pulmonar no microcítico,3,1,3,1
5,Carcinoma pulmonar no microcítico,5,1,0,0
6,Cáncer tiroideo,1,1,0,0
7,Cáncer gástrico,0,0,1,1


In [262]:
Tabla3Mutaciones = pd.DataFrame({'Mutaciones detectadas':mut,'Porcentaje de frecuencia alélica (ADN)': frecuencias_totales})

In [263]:
Tabla3Mutaciones

,Mutaciones detectadas,Porcentaje de frecuencia alélica (ADN)
0,[MYC],[49.56]
1,[KRAS],[66.50]
2,[],[]
3,"[FGFR1, PIK3CA]",[]
4,"[CDK6, MET]","[21.75, 21.75]"
5,"[CTNNB1, EGFR]","[27.51, 35.40]"
6,[HRAS],[46.17]
7,"[KIT, MYC]","[53.12, 60.64]"


In [126]:
#Tabla1Pacientes = pd.DataFrame({'Número de chip': chip, 'Número de paciente': paciente_Data, 'NHC': NHC_Data, 'Número de biopsia': Nbiopsia_Data, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fecha_Data, 'Diagnóstico': texto_Data})
#Tabla2Diagnostico = pd.DataFrame({'Diagnóstico': texto_Data, 'Número de diagnóstico': Ndiagnostico_Data})
Tabla3Mutaciones = pd.DataFrame({'Mutaciones detectadas':mut,'Porcentaje de frecuencia alélica (ADN)': frecuencias_totales,'Variaciones del número de lecturas (ARN)': variaciones_Data,'Variaciones de copias': copias_Data})
#Tabla4Patologicas = pd.DataFrame({'Genes patológicos': patologicos, 'Número de la mutación específica':numero_iden)
#Tabla5MutacionesEspecificas = pd.DataFrame({'Mutaciones detectadas':mut, 'Total del número de mutaciones': num_mutaciones,'Nº de la mutación específica detectada': NmutacionEspecifica_Data})
#Tabla6Informacion = pd.DataFrame({'Diagnóstico': texto_Data, 'Ensayos clínicos': ensayo_Data, 'Fármaco aprobado': farmaco_Data})

NameError: name 'todas_mutaciones' is not defined

In [142]:
joined_df = Tabla1Pacientes.join(Tabla2Diagnostico.set_index('Número de chip', 'Número de paciente'))
joined_df

C:\Users\luciv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only
  """Entry point for launching an IPython kernel.


TypeError: '<' not supported between instances of 'int' and 'str'

In [145]:
# Unir los dataframes utilizando dos columnas como clave primaria
df_merged = Tabla1Pacientes.join(Tabla2Diagnostico.set_index(['Número de chip', 'Número de paciente']), on=['Número de chip', 'Número de paciente'])

# Imprimir el dataframe resultante
print(df_merged)


ValueError: columns overlap but no suffix specified: Index(['Diagnóstico'], dtype='object')

In [96]:
TablaFinal3 = pd.merge(Tabla1Pacientes, Tabla2Diagnostico, on='Diagnóstico', how='outer')
TablaFinal3.drop_duplicates()

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Mutaciones detectadas,Número del diagnóstico
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"['MYC', 'EML4-ALK']",15.1
6,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,['KRAS'],15.1
12,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,[],15.1
18,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"['FGFR1', 'PIK3CA']",15.1
24,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,"['CDK6', 'MET']",15.1
30,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,"['CTNNB1', 'EGFR']",15.1
36,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,['HRAS'],30.0
37,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,"['KIT', 'MYC']",11.0


In [85]:
tablaIdeal = pd.DataFrame({'Número de chip': chip2, 'Núme de paciente': numero_paciente, 'NHC': NHC_final, 'Número de biopsia': NB_values, 'Biopsia sólida': Biopsia_solida, 'Fecha de informe': fechas, 'Diagnóstico': textoDiag, 'Mutaciones detectadas':mut,'Diagnóstico': textoDiag, 'Número del diagnóstico': numeroDiag, 'Diagnóstico': textoDiag, 'Ensayos clínicos': lista_ensayos,'SI/NO ensayo':ensayos_finales, 'Fármaco aprobado': lista_tratamientos, 'SI/NO fármacos': tratamientos_finales })
tablaIdeal

,Número de chip,Núme de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Mutaciones detectadas,Número del diagnóstico,Ensayos clínicos,SI/NO ensayo,Fármaco aprobado,SI/NO fármacos
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"[MYC, EML4-ALK]",15.1,6,1,6,1
1,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,[KRAS],15.1,5,1,1,1
2,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,[],15.1,0,0,0,0
3,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"[FGFR1, PIK3CA]",15.1,0,0,0,0
4,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,"[CDK6, MET]",15.1,3,1,3,1
5,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,"[CTNNB1, EGFR]",15.1,5,1,0,0
6,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,[HRAS],30.0,1,1,0,0
7,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,"[KIT, MYC]",11.0,0,0,1,1


In [86]:
Tabla1Pacientes['Mutaciones detectadas'] = Tabla1Pacientes['Mutaciones detectadas'].astype(str)
tabla_final = pd.merge(Tabla1Pacientes, Tabla2Diagnostico, on='Diagnóstico', how='outer')
tabla_final = tabla_final[~tabla_final.duplicated()]
tabla_final

,Número de chip,Número de paciente,NHC,Número de biopsia,Biopsia sólida,Fecha de informe,Diagnóstico,Mutaciones detectadas,Número del diagnóstico
0,100,1,00021,23B00021-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"['MYC', 'EML4-ALK']",15.1
6,100,2,00022,22B00022-A2,1,24-mar-2023,Carcinoma pulmonar no microcítico,['KRAS'],15.1
12,100,3,00023,21B00023-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,[],15.1
18,100,4,00024,23B00024-A1,1,24-mar-2023,Carcinoma pulmonar no microcítico,"['FGFR1', 'PIK3CA']",15.1
24,100,5,00025,23C00025-A1,3,24-mar-2023,Carcinoma pulmonar no microcítico,"['CDK6', 'MET']",15.1
30,100,6,00026,23P00026,2,24-mar-2023,Carcinoma pulmonar no microcítico,"['CTNNB1', 'EGFR']",15.1
36,100,7,00027,22B00027-C4,1,24-mar-2023,Cáncer tiroideo,['HRAS'],30.0
37,100,8,00028,23B00028-A2,1,24-mar-2023,Cáncer gástrico,"['KIT', 'MYC']",11.0
